In [1]:
from el_toolkit.document import Document
from el_toolkit.lkb.lexical_knowledge_base import Lexical_Knowledge_Base
from transformers import BertTokenizer


lkb = Lexical_Knowledge_Base.read_json("data/knowledge_datasets/mesh_subset.json")

biomed_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='monologg/biobert_v1.1_pubmed',
                                            do_lower_case=False, cache_dir=None)
                                            
doc = Document.read_json("/Users/coltonflowers/Work/TREND_repos/BioMedical-EL/tests/test_data/test_docs/without_overlaps/14_mentions.json")
doc.jupyter_display(lkb)
for segmented_doc in doc.segment(biomed_tokenizer,8):
  segmented_doc.jupyter_display(lkb)
  print("-----------------------------------")

Mentions Omitted: 0


-----------------------------------


-----------------------------------


In [2]:
from el_toolkit.entity_linkers.dual_embedder.featurizer import BertDualEmbedderTrainFeaturizer
from el_toolkit.entity_linkers.dual_embedder.concept_embedder import BertConceptEmbedder
from el_toolkit.entity_linkers.dual_embedder.document_embedder import DocumentEmbedder
from el_toolkit.entity_linkers.dual_embedder.entity_linker import DualEmbedderEntityLinker
from el_toolkit.entity_linkers.dual_embedder.model import BertMentionDetectorModel
from el_toolkit.entity_linkers.dual_embedder.model import BertDualEmbedderModel
from transformers import BertModel
import copy

bert_mention = BertModel.from_pretrained("monologg/biobert_v1.1_pubmed")
bert_candidate = copy.deepcopy(bert_mention)
concept_embedder = BertConceptEmbedder(lkb,bert_model=bert_mention,tokenizer=biomed_tokenizer,max_seq_len=256)
span_detector = BertMentionDetectorModel(bert_mention)
document_embedder = DocumentEmbedder(span_detector=span_detector,tokenizer=biomed_tokenizer,max_seq_len=256)
dual_embedder_model = BertDualEmbedderModel(bert_candidate,bert_mention)
dual_embedder = DualEmbedderEntityLinker(concept_embedder,document_embedder,dual_embedder_model)
featurizer = BertDualEmbedderTrainFeaturizer(lkb,dual_embedder)

Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
doc

doc_id:14_mentions
--------------------
message:Diffuse skeletal pain after administration of alendronate. BACKGROUND: Osteoporosis is caused by bone resorption in excess of bone formation, and bisphosphonates, are used to inhibit bone resorption. Alendronate, a biphosphonate, is effective for both the treatment and prevention of osteoporosis in postmenopausal women. Side effects are relatively few and prominently gastrointestinal. Musculoskeletal pain may be an important side effect in these patients. We presented a patient admitted to our out-patient clinic with diffuse skeletal pain after three consecutive administration of alendronate. CONCLUSION: We conclude that patients with osteoporosis can report pain, and bisphosphonate-related pain should also be considered before ascribing this complaint to osteoporosis.
--------------------
mentions:[{'start_index': 17, 'end_index': 21, 'concept_id': 'D010146', 'text': 'pain'}, {'start_index': 46, 'end_index': 57, 'concept_id': 'D019386', 

In [4]:
featurizer.TrainingInputFeatures._fields

('doc_token_ids',
 'doc_token_masks',
 'mention_start_indices',
 'mention_end_indices',
 'label_ids',
 'num_mentions',
 'candidate_token_ids',
 'candidate_token_masks',
 'candidate_masks')

In [5]:
docs = doc.segment(biomed_tokenizer,8) 
dual_embedder.train(docs,num_random_negatives=4)

Mentions Omitted: 0


/Users/coltonflowers/Work/TREND_repos/BioMedical-EL/el_toolkit/entity_linkers/dual_embedder/model.py:178: UserWarning: __rfloordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  n_c = self.get_number_of_candidates(**kwargs) // num_mentions


torch.Size([8, 768])
torch.Size([8, 5, 768])


RuntimeError: shape '[-1, 6, 768]' is invalid for input of size 30720

In [8]:
dataset= dual_embedder._train_featurizer.featurize(docs,num_random_negatives=4)
dataset[0][0].shape

pain,5,5
alendronate,9,12
Osteoporosis,22,26
bisphosphonates,41,45
Alendronate,57,60
osteoporosis,78,82
Musculoskeletal pain,104,110
pain,137,137
alendronate,6,9
osteoporosis,22,26
pain,29,29
bisphosphonate,32,36
pain,39,39
osteoporosis,50,54


torch.Size([256])

In [9]:
dataset[0][3]

tensor([  5,  12,  26,  45,  60,  82, 110, 137])

In [10]:
dataset[0][2]

tensor([  5,   9,  22,  41,  57,  78, 104, 137])

In [25]:
doc_token_ids,doc_token_mask,mention_start_indices,mention_end_indices,_,_  = dual_embedder.document_embedder.encode_document(doc)

In [6]:
lkb = Lexical_Knowledge_Base.read_json("data/knowledge_datasets/trend.json")